# RAG Agent

Retrieval Augmented Generation (RAG) is one of the most useful applications of AI agents. 

It consists in grounding the answers of our agent to a given knowledge base, that our agent can access by using tools.

This knowledge base - i.e., our collection of documents - is embedded in a vector store for efficient search. 

This method prevents allucinations and highly increases the accuracy of the agentic system.

Steps in this implementation:

1. get a knowledge base: we start with a collection of scientific papers, manually selected;

2. perform OCR of our documents to best extract information;

3. embed our documents in a vector database (vector store)

4. construct a tool for searching in the database

5. create the graph with grading and retries. 

## 1. Knowledge Base

We selected scientific papers on the topic on the topic **"LLM-Agents for Urban Mobility & Traffic Engineering"**.

We stored them in the folder `documents/`.

## 2. OCR (Optical Character Recognition)

As we said, we usually need to perform OCR on a knowledge based composed by documents. 

>Notice that we may skip this step if our knowledge base is already composed of plain text (for example, this could happen if our base is made thorugh web scarping, which usually returns plain text or markdown).

In this case our papers are complex, and contain mathematical expressions, tables, images. OCR is not a simple/skippable step here. 

That's why we are going to user the best OCR model around this days (January 2026): Mistral OCR 3. Find a usage example in the [mistral_ocr](./mistral_ocr.ipynb) notebook.


In [ ]:
import base64
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()
api_key = os.environ["MISTRAL_API_KEY"]

client = Mistral(api_key=api_key)

def encode_pdf(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        return base64.b64encode(pdf_file.read()).decode('utf-8')


pdf_paths = [path for path in os.listdir("documents/") if path.endswith(".pdf")]
for i, pdf_path in enumerate(pdf_paths):
    print(f"Processing document {i}/{len(pdf_paths)}")
    base64_pdf = encode_pdf(pdf_path)

    print("Calling Mistral OCR...")
    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": f"data:application/pdf;base64,{base64_pdf}" 
        },
        # table_format="html", # default is None
        # extract_header=True, # default is False
        # extract_footer=True, # default is False
        include_image_base64=True
    )   

    # parse the response and save the text in raw_text/

Embed data **and metadata** in vector store (so when we retrieve we give the page and document source (maybe we also show it as well! cool))